<a href="https://colab.research.google.com/github/hyunjaecho94/deepmutation/blob/master/c10_flip_layer_order.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import torch, torchvision, matplotlib
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch.nn as nn
torch.manual_seed(0)

from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np
np.random.seed(0)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [14]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [15]:
# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

# CIFAR10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='CIFAR10/data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.CIFAR10(root='CIFAR10/data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)


0it [00:00, ?it/s]

170500096it [00:06, 26508879.04it/s]                               


Extracting CIFAR10/data/cifar-10-python.tar.gz to CIFAR10/data/


In [0]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.fc = nn.Linear(1024, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [25]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/500], Loss: 1.4307
Epoch [1/5], Step [200/500], Loss: 1.2365
Epoch [1/5], Step [300/500], Loss: 1.0135
Epoch [1/5], Step [400/500], Loss: 1.2231
Epoch [1/5], Step [500/500], Loss: 1.0237
Epoch [2/5], Step [100/500], Loss: 1.0376
Epoch [2/5], Step [200/500], Loss: 1.1406
Epoch [2/5], Step [300/500], Loss: 1.1295
Epoch [2/5], Step [400/500], Loss: 0.8681
Epoch [2/5], Step [500/500], Loss: 0.9812
Epoch [3/5], Step [100/500], Loss: 0.9466
Epoch [3/5], Step [200/500], Loss: 0.8832
Epoch [3/5], Step [300/500], Loss: 1.1652
Epoch [3/5], Step [400/500], Loss: 1.1248
Epoch [3/5], Step [500/500], Loss: 0.9385
Epoch [4/5], Step [100/500], Loss: 1.0430
Epoch [4/5], Step [200/500], Loss: 0.8081
Epoch [4/5], Step [300/500], Loss: 0.7621
Epoch [4/5], Step [400/500], Loss: 0.7547
Epoch [4/5], Step [500/500], Loss: 0.8847
Epoch [5/5], Step [100/500], Loss: 0.8118
Epoch [5/5], Step [200/500], Loss: 0.9334
Epoch [5/5], Step [300/500], Loss: 0.7551
Epoch [5/5], Step [400/500], Loss:

In [0]:
path = F"/content/gdrive/My Drive/correct_test_dataset.txt"
correct_test = np.loadtxt(path, delimiter=",", dtype=int)

In [0]:
# Dictionary to record which indices are incorrectly classified
# incorrection_dict[true_label] = what model thinks the class is
incorrection_dict = {}
incorrection_dict[0] = []
incorrection_dict[1] = []
incorrection_dict[2] = []
incorrection_dict[3] = []
incorrection_dict[4] = []
incorrection_dict[5] = []
incorrection_dict[5] = []
incorrection_dict[6] = []
incorrection_dict[7] = []
incorrection_dict[8] = []
incorrection_dict[9] = []

In [28]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    for idx, (images, labels) in enumerate(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        #only test ones correctly classified by the original model
        for j in range(len(predicted)):
          if idx*batch_size+j in correct_test:
            if labels[j] == predicted[j]:
              correct +=1
            else:
              incorrection_dict[labels[j].item()].append(predicted[j].item())

    total = len(correct_test)
        
    print('Test Accuracy of the model on the {} test images (100% for original model): {} %'.format(total, 100 * correct / total))


Test Accuracy of the model on the 9896 test images (100% for original model): 57.679870654810024 %


In [29]:
print(correct/total)

0.5767987065481003


In [30]:
incorrection_dict

{0: [6,
  8,
  8,
  8,
  8,
  1,
  8,
  8,
  2,
  8,
  6,
  3,
  8,
  1,
  3,
  8,
  3,
  8,
  3,
  1,
  8,
  8,
  8,
  8,
  8,
  3,
  3,
  3,
  8,
  9,
  8,
  3,
  3,
  8,
  8,
  8,
  1,
  2,
  8,
  1,
  3,
  8,
  8,
  8,
  8,
  2,
  1,
  8,
  2,
  2,
  2,
  1,
  2,
  1,
  8,
  3,
  3,
  8,
  8,
  8,
  2,
  8,
  1,
  8,
  8,
  8,
  2,
  8,
  8,
  8,
  8,
  3,
  1,
  2,
  2,
  8,
  1,
  8,
  1,
  8,
  8,
  8,
  1,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  2,
  9,
  8,
  3,
  8,
  8,
  3,
  8,
  8,
  8,
  8,
  8,
  1,
  8,
  3,
  2,
  3,
  8,
  1,
  3,
  2,
  8,
  8,
  3,
  8,
  4,
  1,
  8,
  8,
  8,
  1,
  1,
  1,
  8,
  9,
  8,
  3,
  2,
  8,
  9,
  8,
  8,
  8,
  9,
  8,
  8,
  1,
  8,
  8,
  8],
 1: [0,
  8,
  0,
  0,
  8,
  0,
  0,
  0,
  0,
  8,
  8,
  0,
  8,
  0,
  8,
  8,
  0,
  0,
  8,
  8,
  8,
  8,
  0,
  8,
  0,
  0,
  0,
  8,
  8,
  0,
  0,
  3,
  8,
  8,
  8,
  0,
  0,
  0,
  0,
  8,
  8,
  0,
  8,
  3,
  8,
  9,
  0,
  0,
  8,
  8,
  8,
  0,
  8,
  8,
  5,
  8,
  0

In [0]:
path =F"/content/gdrive/My Drive/correction_dict.npy" 
correction_dict=np.load(path, allow_pickle=True)

In [32]:
def calc_mutation_score():
  score = 0
  for key, value in incorrection_dict.items():
    count_misclassify = len(value)
    normalizing = len(correction_dict.item().get(key))
    score += count_misclassify/normalizing
  return score
calc_mutation_score()

4.3027056569756335